In [ ]:
!pip3 install beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup

In [42]:
nome = "BBC"
busca = "https://www.bbc.com/portuguese/search/?q=marielle+franco&start=<pagina>"
description = "BBC News é a seção de Jornalismo dentro da BBC, atuante em TV, rádio e Internet e diversas partes do mundo, inclusive no Brasil."
lista_noticias = []
max_paginas = 2

for i in range(max_paginas):
    page = requests.get(busca.replace("<pagina>", str(i+1)))
    soup = BeautifulSoup(page.text, 'html.parser')
    lista_noticias += [x['href'] for x in soup.find_all(class_="hard-news-unit__headline-link")]

noticias = []
for link in lista_noticias:
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    noticias.append({
        'Título': soup.find(class_="story-body__h1").text,
        'Autor': soup.find(class_='byline__name').text if soup.find(class_='byline__name') else "",
        'Endereço': link,
        'Seção': soup.find("meta", property="article:section")['content'] if soup.find("meta", property="article:section") else "",
        'Data de publicação': soup.find(class_='date date--v2').text,
        'Sobre a mídia': description,
        'Lide': soup.find(class_="story-body__introduction").text,
        'Texto': soup.find(class_='story-body__inner').get_text(),
    })

for i, noticia in enumerate(noticias):
    with open("BBC-" + str(i+1) + ".txt", "w") as f:
        f.write("\n".join(["# " + x + " = " + noticia[x] for x in noticia if x != "Texto"]) + "\n" + noticia["Texto"])